![Dask Icon](dask_horizontal_black.gif "Dask Icon")
![Pandas Icon](images/pandas_logo.png "Pandas Icon")

# Gotcha's from Pandas to Dask

This notebook highlights some key differences when transfering code from `Pandas` to run in a `Dask` environment.  
Most issues have a link to the [Dask documentation](https://docs.dask.org/en/latest/) for additional information.

# Agenda  
1. Intro to `Dask` Dataframe
2. Basic Comparison
3. 

In [3]:
# since Dask is activly beeing developed - the current example is running with the below version
import dask
import pandas as pd
print(f'Dask versoin: {dask.__version__}')
print(f'Pandas versoin: {pd.__version__}')

Dask versoin: 1.2.2
Pandas versoin: 0.24.2


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  In this example we are running on a `LocalCluster`, this  will also provide a dashboard which is useful to gain insight on the computation.  
For additional information on [Dask Client see documentation](https://docs.dask.org/en/latest/setup.html?highlight=client#setup)  

The link to the dashboard will become visible when you create a client (as shown below).  
When running in `Jupyter Lab` an [extenstion](https://github.com/dask/dask-labextension) can be installed to be able to view the various dashboard widgets. 

In [4]:
from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client()
client

c:\users\jsber\.virtualenvs\dask_pyconil2019-dwj2t3bv\lib\site-packages\distributed\bokeh\core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:50494 Dashboard: http://127.0.0.1:50497/status,Cluster Workers: 4 Cores: 4 Memory: 8.50 GB


See [documentation for addtional cluster configuration](http://distributed.dask.org/en/latest/local-cluster.html)

* When running code within a script use `context manager`  
see question in [stack overflow](https://stackoverflow.com/a/53520917/5817977)  
* In order to get url dashboard use [inner function ](https://github.com/dask/distributed/issues/2083#issue-337057906)  



```python   
with Client() as client:
    ...
```

# Create 2 DataFrames for comparison: 
1. for Dask 
2. for Pandas  
Dask comes with builtin dataset samples, we will use this sample for our example. 

In [109]:
ddf = dask.datasets.timeseries()
print(type(ddf))
ddf

<class 'dask.dataframe.core.DataFrame'>


,id,name,x,y
npartitions=30,,,,
2000-01-01,int32,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


* Remember `Dask framework` is **lazy** thus in order to see the result we need to run [compute()](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.compute) 
 (or `head()` which runs under the hood compute()) )

In [6]:
ddf.head(2)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1003,Norbert,-0.949590,-0.794612
2000-01-01 00:00:01,1038,George,-0.724999,0.361636


 ## Consider using Persist
Since Dask is lazy - it may run the **entire** graph/DAG (again) even if it already run part of the calculation in a previous cell.  Thus use [persist](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#persist-intelligently) to keep the results in memory 
```python
ddf = client.persist(ddf)
```
This is different from Pandas which once a variable was created it will keep all data in memory.  
Additional information can be read in this [stackoverflow issue](https://stackoverflow.com/questions/45941528/how-to-efficiently-send-a-large-numpy-array-to-the-cluster-with-dask-array/45941529#45941529) or see an exampel in [this post](http://matthewrocklin.com/blog/work/2017/01/12/dask-dataframes)   
This concept should also  be used when running a code within a script (rather then a jupyter notebook) which incoperates loops within the code.

In [112]:
ddf = dask.datasets.timeseries()
ddf = client.persist(ddf)
ddf.head(2)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,986,Laura,0.384071,-0.066949
2000-01-01 00:00:01,1035,Quinn,0.916566,0.384282


## Pandas
In order to create a `Pandas` dataframe we can use the `compute()` method from a `Dask dataframe`

In [8]:
pdf = ddf.compute()  
print(type(pdf))
pdf.head(2)

<class 'pandas.core.frame.DataFrame'>


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,954,George,-0.675822,0.384542
2000-01-01 00:00:01,985,Laura,-0.255242,0.804687


## Creating a `Dask dataframe` from `Pandas`
In order to utilize `Dask` capablities on an existing `Pandas dataframe` (pdf) we need to convert the `Pandas dataframe` into a `Dask dataframe` (ddf)  with the [from_pandas](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_pandas) method. 
You must supply the number of partitions or chunksize that will be used to generate the dask dataframe

In [61]:
ddf2 = dask.dataframe.from_pandas(pdf, npartitions=10)
ddf2

,id,name,x,y
npartitions=10,,,,
2000-01-01 00:00:00,int32,object,float64,float64
2000-01-04 00:00:00,...,...,...,...
...,...,...,...,...
2000-01-28 00:00:00,...,...,...,...
2000-01-30 23:59:59,...,...,...,...


## Partitions in Dask Dataframes

Notice that when we created a `Dask dataframe` we needed to supply an argument of `npartitions`.  
The number of partitions will assist `Dask` on how it's going to parallelize the computation.  
Each partition is a *separate* dataframe. For additional information see [partition documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#partitions)  

An example for this can be seen when examing the `reset_ index()` method:

In [21]:
pdf2 = pdf.reset_index()
# Only 1 row
pdf2.iloc[0]

timestamp    2000-01-01 00:00:00
id                           954
name                      George
x                      -0.675822
y                       0.384542
Name: 0, dtype: object

In [34]:
ddf2 = ddf2.reset_index()
# each partition has an index=0
ddf2.loc[0].compute() 
# ddf2.loc[0].visualize()

,timestamp,id,name,x,y
0,2000-01-01,954,George,-0.675822,0.384542
0,2000-01-04,1015,Wendy,0.227670,-0.697287
0,2000-01-07,979,Victor,0.292172,0.272159
0,2000-01-10,954,Dan,0.646783,-0.810418
0,2000-01-13,984,Victor,0.384245,0.530561
0,2000-01-16,1023,Quinn,-0.656461,-0.062047
0,2000-01-19,983,Victor,-0.406623,0.409387
0,2000-01-22,942,Laura,-0.058315,0.138405
0,2000-01-25,1021,Alice,-0.354155,-0.656652
0,2000-01-28,976,Dan,-0.355238,0.615423


Now that we have a `dask` (ddf) and a `pandas` (pdf) dataframe we can start to compair the interactions with them.

In [113]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,986,Laura,0.384071,-0.066949
2000-01-01 00:00:01,1035,Quinn,0.916566,0.384282
2000-01-01 00:00:02,967,Charlie,0.922719,-0.556339
2000-01-01 00:00:03,1005,Hannah,0.461346,0.977012
2000-01-01 00:00:04,1008,Wendy,0.848642,0.793633


# Conceptual shift - from Update to Insert/Delete
Dask does not update - thus there are no arguments such as `inplace=True` which exist in Pandas.  
For more detials see [issue#653 on github](https://github.com/dask/dask/issues/653)

### Rename Columns

* using `inplace=True` is not considerd to be *best practice*. 

In [19]:
# Pandas 
print(pdf.columns)
pdf.rename(columns={'id':'ID'}, inplace=True)
# pdf = pdf.rename(columns={'id':'ID'})
pdf.columns

Index(['ID', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

In [20]:
# Dask - Error
ddf.rename(columns={'id':'ID'}, inplace=True)
ddf.columns

TypeError: rename() got an unexpected keyword argument 'inplace'

In [21]:
# Dask
print(ddf.columns)
ddf = ddf.rename(columns={'id':'ID'})
ddf.columns

Index(['ID', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

## Data munipilations  
There are several diffrences when manipulating data.  

### loc - Pandas

In [22]:
cond = (pdf['x']>0.5) &(pdf['x']<0.8)
pdf[cond].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:03,1025,Bob,0.799201,-0.503440
2000-01-01 00:00:05,996,Ursula,0.592114,0.104375


In [27]:
pdf.loc[cond, ['y']] = pdf['y']* 100
pdf[cond].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:03,1025,Bob,0.799201,-5034.404280
2000-01-01 00:00:05,996,Ursula,0.592114,1043.753529


### Dask - use mask/where

In [25]:
cond_dask = (ddf['x']>0.5) & (ddf['x']<0.8)

# Error
ddf.loc[cond_dask, ['y']] = ddf['y']* 100

TypeError: '_LocIndexer' object does not support item assignment

In [28]:
ddf['y'] = ddf['y'].mask(cond_dask, ddf['y']* 100)
ddf[cond_dask].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:03,1025,Bob,0.799201,-5034.404280
2000-01-01 00:00:05,996,Ursula,0.592114,1043.753529


## Meta
One key difference is the introduction of `meta` arguement.  
> `meta` is the prescription of the names/types of the output from the computation  
[see stack overflow answer](https://stackoverflow.com/questions/44432868/dask-dataframe-apply-meta)

Since `Dask` creates a DAG for the computation it requires to understand what are the outputs of each calculation.  
For additinal information see [meta documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#metadata)

In [29]:
pdf['initials'] = pdf['name'].apply(lambda x: x[0]+x[1])
pdf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La


In [30]:
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1])
ddf.head(2)

/home/ds/.local/share/virtualenvs/dask_pyconil2019-9doxB0Ra/lib/python3.7/site-packages/dask/dataframe/core.py:2345: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('name', 'object'))

  warnings.warn(meta_warning(meta))


,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La


In [31]:
# Describe the outcome type of the calculation
meta_cal = pd.Series(object, name='initials')

In [32]:
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1], meta = meta_cal)
ddf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La


In [33]:
def func(row):
    if (row['x']> 0):
        return row['x'] * 1000  
    else:
        return row['y'] * -1

In [34]:
# ddf['z'] = ddf.apply(func, args=('coor_x', 'coor_y'), axis=1, meta=('z', 'float'))
ddf['z'] = ddf.apply(func, axis=1, meta=('z', 'float'))
ddf.head(2)

,ID,name,x,y,initials,z
timestamp,,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa,141.229048
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La,427.486691


### Map partitions
* We can supply a ad-hoc function to run on each partition using the [map_partitions](https://dask.readthedocs.io/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions) method.   
Mainly useful for functions that are not implemented in `Dask` or `Pandas` .  
The function could also include arguments.

In [104]:
import numpy as np
def func2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist

In [110]:
ddf['dist'] = ddf.map_partitions(func2, coor_x='x', coor_y='y', meta=('float'))

In [111]:
ddf.head(2)

,id,name,x,y,dist
timestamp,,,,,
2000-01-01 00:00:00,1000,Laura,-0.054607,-0.411219,NaN
2000-01-01 00:00:01,1001,Wendy,-0.552326,-0.548432,0.516286


* Finally we can return a new `dataframe` which needs to be described in the `meta` argument

In [48]:
def func3(df, col1, col2, col3, col4):
    df['col12'] = df[col1] * df[col2]
    df = df.drop([col3, col4], axis=1)
    return df

In [49]:
ddf_z = ddf.map_partitions(func3, 'x', 'y', 'z','a',
                           meta=pd.DataFrame({'ID':'i8'
                                              , 'name':str
                                              , 'x':'f8'
                                              ,'y':'f8'
                                              , 'initials':str
                                              , 'col12':'f8'}, index=[0]) 
                          )
ddf_z.head(2)

,ID,name,x,y,initials,col12
timestamp,,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa,-0.011128
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La,0.311925


### Convert index into Time column

In [50]:
# Pandas
pdf = pdf.assign(Time=pd.to_datetime(pdf.index).time)
pdf.head(2)

,ID,name,x,y,initials,Time
timestamp,,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa,00:00:00
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La,00:00:01


In [144]:
# ddf = ddf.assign(Time= dask.dataframe.to_datetime(ddf.index, format='%Y-%m-%d'). )
ddf = ddf.assign(Time= dask.dataframe.to_datetime(ddf.index).dt.time )
ddf.head(2)

,id,name,x,y,col,Time
timestamp,,,,,,
2000-01-01 00:00:00,1036,Xavier,-0.514155,0.887133,NaN,00:00:00
2000-01-01 00:00:01,969,Victor,-0.263525,-0.289397,1.202929,00:00:01


In [52]:
# Dask or Pandas
ddf = ddf.assign(Time2=ddf.index)
ddf['Time2'] = ddf['Time2'].dt.time
ddf.head()

,ID,name,x,y,initials,z,a,Time,Time2
timestamp,,,,,,,,,
2000-01-01 00:00:00,964,Patricia,0.141229,-0.078796,Pa,141.229048,-0.011128,00:00:00,00:00:00
2000-01-01 00:00:01,1006,Laura,0.427487,0.729673,La,427.486691,0.311925,00:00:01,00:00:01
2000-01-01 00:00:02,983,Bob,0.465053,-0.613880,Bo,465.052706,-0.285487,00:00:02,00:00:02
2000-01-01 00:00:03,1025,Bob,0.799201,-5034.404280,Bo,799.200722,-4023.499537,00:00:03,00:00:03
2000-01-01 00:00:04,1061,Wendy,-0.768113,-0.022926,We,0.022926,0.017610,00:00:04,00:00:04


## Drop NA on column

In [53]:
# Pandas
pdf = pdf.assign(colna = None)
print(pdf.head(2))
pdf.dropna(axis=1, how='all', inplace=True)
print(pdf.head(2))

                       ID      name         x         y initials      Time  \
timestamp                                                                    
2000-01-01 00:00:00   964  Patricia  0.141229 -0.078796       Pa  00:00:00   
2000-01-01 00:00:01  1006     Laura  0.427487  0.729673       La  00:00:01   

                    colna  
timestamp                  
2000-01-01 00:00:00  None  
2000-01-01 00:00:01  None  
                       ID      name         x         y initials      Time
timestamp                                                                 
2000-01-01 00:00:00   964  Patricia  0.141229 -0.078796       Pa  00:00:00
2000-01-01 00:00:01  1006     Laura  0.427487  0.729673       La  00:00:01


In odrer for `Dask` to drop a column with all `na` 

In [61]:
# Dask
ddf = ddf.assign(colna = None)
print(ddf.head(2))

ddf = ddf.dropna(how='all')

# if ddf.colna.isnull().all() == True:   # check if all values in column are Null - VERY slow
#     ddf = ddf.drop(labels=['colna'],axis=1)
print(ddf.head(2))

                       ID      name         x         y initials           z  \
timestamp                                                                      
2000-01-01 00:00:00   964  Patricia  0.141229 -0.078796       Pa  141.229048   
2000-01-01 00:00:01  1006     Laura  0.427487  0.729673       La  427.486691   

                            a      Time     Time2 colna  
timestamp                                                
2000-01-01 00:00:00 -0.011128  00:00:00  00:00:00  None  
2000-01-01 00:00:01  0.311925  00:00:01  00:00:01  None  
                       ID      name         x         y initials           z  \
timestamp                                                                      
2000-01-01 00:00:00   964  Patricia  0.141229 -0.078796       Pa  141.229048   
2000-01-01 00:00:01  1006     Laura  0.427487  0.729673       La  427.486691   

                            a      Time     Time2 colna  
timestamp                                                
2000-01-01

##  Reset Index

In [ ]:
# Pandas
pdf.reset_index(drop=True, inplace=True)
pdf.head(2)

In [ ]:
# Dask
ddf = ddf.reset_index()
ddf['Time'] = ddf['timestamp'].dt.time
ddf = ddf.drop(labels=['timestamp'], axis=1 )
ddf.columns = ['ID','name','x','y','Time']
ddf.head(2)

# Read / Save files

When working with `pandas` and `dask` preferable try and work with [parquet](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#store-data-in-apache-parquet-format).  
Even so when working with `Dask` - the files can be read with multiple workers .  
Most `kwargs` are applicable for reading and writing files [see documentaion](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.to_csv) (including the option for output file naming).  
e.g. 
ddf = dask.dataframe.read_csv('data/pd2dd/ddf*.csv', compression='gzip', header=False).  

However some are not available such as  `nrows`.

## Save files

In [ ]:
# Pandas
!mkdir data
pdf.to_csv('data/pdf_single_file.csv')

In [ ]:
!dir data  # use ls on linux systems

`Dask`
Notice the '*' to allow for multiple file renaming. 



In [ ]:
# Dask
!mkdir data\pd2dd
ddf.to_csv('data/pd2dd/ddf*.csv', index = False)

In [ ]:
!dir data\pd2dd\ 

To find the number of partitions which will determine the number of output files use [dask.dataframe.npartitions](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.npartitions)  

To change the number of output files use [repartition](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.repartition) which is an expensive operation.

## Read files

For `pandas` it is possible to iterate and concat the files [see answer from stack overflow](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe).

In [ ]:
# Pandas 
import glob
import os

path = r'data/pd2dd/'                     # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
len(concatenated_df)

In [ ]:
# Dask
_ddf = dask.dataframe.read_csv('data/pd2dd/ddf*.csv')
len(_ddf)

# Group By - custom aggregations
In addition to the [groupby notebook example](https://github.com/dask/dask-examples/blob/master/dataframes/02-groupby.ipynb)  - 
this is another example how to try to eliminate the use of `groupby.apply`  
In this example we are grouping by columns into unique list.

In [136]:
# prepare pandas dataframe
pdf = pdf.assign(Time=pd.to_datetime(pdf.index).time)
pdf['seconds'] = pdf.Time.astype(str).str[-2:]
pdf.head()

,id,name,x,y,Time,seconds
timestamp,,,,,,
2000-01-01 00:00:00,954,George,-0.675822,0.384542,00:00:00,00
2000-01-01 00:00:01,985,Laura,-0.255242,0.804687,00:00:01,01
2000-01-01 00:00:02,981,Quinn,0.163648,-0.545561,00:00:02,02
2000-01-01 00:00:03,992,George,0.298219,0.109008,00:00:03,03
2000-01-01 00:00:04,1037,Zelda,-0.712677,-0.365287,00:00:04,04


In [137]:
# pandas preperations
def set_list_att(x: dask.dataframe.Series):
        return list(set([item for item in x.values]))

In [141]:
%%timeit
# pandas option 1 using apply
pdf_gb = pdf.groupby(pdf.name)
gp_col = ['id', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply(set_list_att) for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

2.18 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [142]:
%%timeit
# pandas option 2 using lambda
pdf_gb = pdf.groupby(pdf.name)
gp_col = ['id', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply(lambda x: list(set(x.to_list()))) for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

1.29 s ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In any case sometimes using Pandas is more efficiante (assuming that you can load all the data into the RAM).  
In this case Pandas is faster

In [145]:
# prepare dask dataframe
ddf['seconds'] = ddf.Time.astype(str).str[-2:]
ddf = client.persist(ddf)
ddf.head()

,id,name,x,y,col,Time,seconds
timestamp,,,,,,,
2000-01-01 00:00:00,1036,Xavier,-0.514155,0.887133,NaN,00:00:00,00
2000-01-01 00:00:01,969,Victor,-0.263525,-0.289397,1.202929,00:00:01,01
2000-01-01 00:00:02,996,Quinn,0.749976,0.408172,1.230361,00:00:02,02
2000-01-01 00:00:03,1074,Charlie,0.342294,0.608586,0.454280,00:00:03,03
2000-01-01 00:00:04,1027,Alice,-0.049236,0.310416,0.492140,00:00:04,04


In [147]:
%%timeit
# Dask option1 using apply
# notice the meta argument in the apply function
df_gb = ddf.groupby(ddf.name)
gp_col = ['id', 'seconds']
list_ser_gb = [df_gb[att_col_gr].apply(set_list_att
                                      ,meta=pd.Series(dtype='object', name=f'{att_col_gr}_att')) 
               for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

22 s ± 2.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Using [dask custom aggregation](https://docs.dask.org/en/latest/dataframe-api.html?highlight=dropna#dask.dataframe.groupby.Aggregation) is consideribly better

In [148]:
# Dask
# some preperations
import itertools
custom_agg = dask.dataframe.Aggregation(
    'custom_agg', 
    lambda s: s.apply(set), 
    lambda s: s.apply(lambda chunks: list(set(itertools.chain.from_iterable(chunks)))),
)

In [150]:
%%time
# Dask option1 using apply
df_gb = ddf.groupby(ddf.name)
gp_col = ['id', 'seconds']
list_ser_gb = [df_gb[att_col_gr].agg(custom_agg) for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')        
df_edge_att.head()

Wall time: 4.22 s


## Debugging
Debugging may be more challenging since
1. when using a client - mutliprocessing is complecated
2. sometime introducing a faulty command into a graph (such as in a jupyter notebook) requirues to cache-out the graph and start the process from the begining

## Corrupted DAG

In [131]:
ddf = dask.datasets.timeseries()

In [132]:
# returns an error
def func_dist2(df, coor_x, coor_y):
#     dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  +  (df[coor_y] - df[coor_y].shift())**2 )
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2  +  (df[coor_y] - df[coor_y].shift())^2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y', meta=('float'))
ddf.head(2)

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

* Even if the function is currected the DAG is corrupted

In [134]:
# Still results with an error
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  +  (df[coor_y] - df[coor_y].shift())**2 )
#     dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2  +  (df[coor_y] - df[coor_y].shift())^2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y', meta=('float'))
ddf.head(2)

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

Need to reset the dataframe

In [135]:
ddf = dask.datasets.timeseries()
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  +  (df[coor_y] - df[coor_y].shift())**2 )
#     dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2  +  (df[coor_y] - df[coor_y].shift())^2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y', meta=('float'))
ddf.head(2)

,id,name,x,y,col
timestamp,,,,,
2000-01-01 00:00:00,1036,Xavier,-0.514155,0.887133,NaN
2000-01-01 00:00:01,969,Victor,-0.263525,-0.289397,1.202929
